### 实例描述
#### 先建立一个100大小的队列、主线程使用计数器不同的加1，队列线程再把主线程里的计数器放到队列里。当队列为空时，主线程在sess.run(queue.dequeue())语句位置挂起，当队列线程写入队列中时。主线程的计数器同步开始工作。

In [1]:
import tensorflow as tf

In [2]:
# 创建长度为100的队列
queue = tf.FIFOQueue(100, 'float')

# 计数器
c = tf.Variable(0.0)
# 加1操作
op = tf.assign_add(c, tf.constant(1.0))
# 操作：将计数器的结果加入队列
enqueue_op = queue.enqueue(c)

In [3]:
# 创建一个队列管理器QueueRunner,用这两个操作向q中添加元素。目前我们只是用一个线程
qr = tf.train.QueueRunner(queue, enqueue_ops=[op, enqueue_op])

Instructions for updating:
To construct input pipelines, use the `tf.data` module.


In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 线程的协调器。
    coord = tf.train.Coordinator()
    # 启动入队线程，Coordinator是线程的参数
    enqueue_threads = qr.create_threads(sess, coord=coord, start=True)
    # 主线程
    for i in range(0,10):
        print('------------------')
        print(sess.run(queue.dequeue()))
    # 通知其他线程关闭 其他所有线程关闭之后，这一函数才能返回
    coord.request_stop()

------------------
832.0
------------------
840.0
------------------
849.0
------------------
853.0
------------------
859.0
------------------
868.0
------------------
876.0
------------------
883.0
------------------
892.0
------------------
901.0
